In [1]:
from lvmtipo.actors import lvm
import asyncio

In [2]:
exptime = 5.0

In [3]:
tel = lvm.sci

await tel.start()
await tel.ag.status()


[ERROR]: mismatch between message correlation_id=9e6d83d9-ee07-4ea0-80a8-67e4eb3e04cc and header command_id=b'9e6d83d9-ee07-4ea0-80a8-67e4eb3e04cc' in message IncomingMessage:{'app_id': None,
 'body_size': 61,
 'cluster_id': '',
 'consumer_tag': 'ctag1.10dbc97cb2621cfbd15e9ad21e8d2af4',
 'content_encoding': '',
 'content_type': 'text/json',
 'correlation_id': '9e6d83d9-ee07-4ea0-80a8-67e4eb3e04cc',
 'delivery_mode': 1,
 'delivery_tag': 1,
 'exchange': 'sdss_exchange',
 'expiration': None,
 'headers': {'command_id': b'9e6d83d9-ee07-4ea0-80a8-67e4eb3e04cc',
             'commander_id': b'lvm-jupyter_ipykernel_launcher.py-b2a414f6.lvm.s'
                             b'ci.ag',
             'internal': False,
             'message_code': b'f',
             'sender': b'lvm.sci.ag'},
 'message_id': '8110d5a5cb97825af6b36d9496bc8ae1',
 'priority': 0,
 'redelivered': False,
 'reply_to': None,
 'routing_key': 'reply.lvm-jupyter_ipykernel_launcher.py-b2a414f6',
 'timestamp': None,
 'type': 'None'

IndexError: list index out of range

In [ ]:
# dont forget the focus !

await tel.foc.status()

[ERROR]: mismatch between message correlation_id=29b489a2-5c65-482b-98a6-9d8328106f86 and header command_id=b'29b489a2-5c65-482b-98a6-9d8328106f86' in message IncomingMessage:{'app_id': None,
 'body_size': 64,
 'cluster_id': '',
 'consumer_tag': 'ctag1.43d1490fcfe52a9d0bd5c54700ce1de9',
 'content_encoding': '',
 'content_type': 'text/json',
 'correlation_id': '29b489a2-5c65-482b-98a6-9d8328106f86',
 'delivery_mode': 1,
 'delivery_tag': 16,
 'exchange': 'sdss_exchange',
 'expiration': None,
 'headers': {'command_id': b'29b489a2-5c65-482b-98a6-9d8328106f86',
             'commander_id': b'lvm-jupyter_ipykernel_launcher.py-35b53520.lvm.s'
                             b'ci.agcam',
             'internal': False,
             'message_code': b'f',
             'sender': b'lvm.sci.agcam'},
 'message_id': 'f63f31b4d841f9675007b8304f38f563',
 'priority': 0,
 'redelivered': False,
 'reply_to': None,
 'routing_key': 'reply.lvm-jupyter_ipykernel_launcher.py-35b53520',
 'timestamp': None,
 'type':

In [39]:
ra_h, dec_d = (await tel.pwi.status()).flatten().unpack("ra_j2000_hours", "dec_j2000_degs")
print(f"ra_h:dec_d {ra_h}:{dec_d}")

await tel.km.slewStart(ra_h, dec_d)

ra_h:dec_d 22.959229413523:-29.6242908413143


{'Reachable': True,
 'AtHome': False,
 'Moving': False,
 'PositionSwitchStatus': 254,
 'Position': -61.129777785164805,
 'DeviceEncoder': {'Position': 1329664.0, 'Unit': 'STEPS'},
 'Velocity': 0.0,
 'AtLimit': False,
 'CurrentTime': 1678460538878999.0,
 'Simulate': True,
 'SkyPA': -122.25955555703297}

In [40]:
# ra/dec are optional and currently not used
print(await tel.ag.guideStart(exptime, ra_h, dec_d))
await asyncio.sleep(30.0)
print(await tel.ag.guideStop())

{'state': 'GUIDE', 'reference_position': {'_type': 'astropy.coordinates.SkyCoord', 'frame': 'fk5', 'representation': 'spherical', 'lon': 116.0934634744157, 'lat': -9.552497968628044}, 'reference_filenames': ['/data/lvm/skyw/agcam/west/20230225/lvm.skyw.agcam.west_00000287.fits', '/data/lvm/skyw/agcam/east/20230225/lvm.skyw.agcam.east_00000287.fits']}
{'state': 'IDLE'}
